# Create a cluster of custom shape from a bulk material

Create a cluster with shape provided by condition on coordinates and crystal orientation along the z-axis from a bulk material.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 


## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment


### 1.1. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("create_cluster_custom_shape.ipynb")

### 1.2. Set up cluster parameters

In [ ]:
RADIUS = 0.3  # in crystal units
VACUUM = 10.0  # in Angstroms on each side
SUPERCELL_SIZE = 10  # in crystal units
Z_ORIENTATION = (0, 0, 1)  # Miller indices of the slab orientation along the z-axis for the cluster
NAME = "Icosahedron"  # Name of the cluster

### 1.3. Set coordinates condition

In [ ]:
import numpy as np
from typing import List
from icosphere import icosphere
from mat3ra.made.tools.utils.coordinate import CoordinateCondition


# Example of a custom coordinate condition (icosahedron). Adapt coordinate conditions to your needs.
class CustomCoordinateCondition(CoordinateCondition):
    """Creates an icosahedron shape using the icosphere module"""
    radius: float = 1
    frequency: int = 1
    vertices: List[List[float]] = icosphere(1)[0]
    faces: List[List[int]] = icosphere(1)[1]
    center: List[float] = [0.5, 0.5, 0.5]

    def condition(self, coordinate: List[float]) -> bool:
        """Returns True if point is inside icosahedron"""
        coordinate = np.array(coordinate) - self.center
        for face in self.faces:
            a, b, c = face
            normal = np.cross(self.vertices[b] - self.vertices[a], self.vertices[c] - self.vertices[a])
            normal = normal / np.linalg.norm(normal)
            if np.dot(normal, coordinate) > self.radius:
                return False
        return True
    
    
condition = CustomCoordinateCondition(radius=RADIUS).condition

### 1.3. Get input material

In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

## 2. Create the Target Material
### 2.1. Set the cluster configuration


In [ ]:
from mat3ra.made.tools.build.nanoparticle import ASEBasedNanoparticleConfiguration
from mat3ra.made.tools.build.nanoparticle.enums import ASENanoparticleShapesEnum


config = ASEBasedNanoparticleConfiguration(
    material=materials[0],
    supercell_size=SUPERCELL_SIZE,
    vacuum=VACUUM,
    z_orientation=Z_ORIENTATION,
    coordinate_condition=condition
)


#### 2.2. Create the cluster

In [ ]:
from mat3ra.made.tools.build.nanoparticle import ASEBasedNanoparticleBuilder

### 2.2. Set lattice to Cubic

In [ ]:
from mat3ra.made.lattice import Lattice

current_vector_1, current_vector_2, current_vector_3 = cluster.lattice.vectors
cubic_vector_1 = [current_vector_1[0], 0, 0]
cubic_vector_2 = [0, current_vector_2[1], 0]
cubic_vector_3 = [0, 0, current_vector_3[2]]
cluster.lattice = Lattice.from_vectors_array([cubic_vector_1, cubic_vector_2, cubic_vector_3], type="CUB")

## 3. Visualize the Result(s)

In [ ]:
visualize([{"material": slab, "title": "Original material"},
           {"material": cluster, "title": f"Cluster"}])

visualize([{"material": slab, "title": "Original material"},
           {"material": cluster, "title": f"Cluster"}], rotation="-90x")

## 4. Pass data to the outside runtime

In [ ]:
from utils.jupyterlite import set_materials

cluster.name = f"{materials[0].name} {NAME}"
set_materials(cluster)